In [ ]:
from importlib import reload

import numpy as np
import os

import matplotlib.pyplot as plt
import torch

from diffdrr.drr import DRR
from diffdrr.data import load_example_ct
from diffdrr.visualization import plot_drr

## library to read lung cancer database
from sqlalchemy import func # required to query the db
import thirdparty.pylidc.pylidc as pl

### Read Lung Cancer Database files
Follow instructions at:

https://pylidc.github.io/

When running the code below, if getting errors such as np.bool not found or np.int not found, change the sources as follows

- replace *np.int* by *int*

- replace *np.bool* by *bool*

First we need to set the path to the LIDC Dataset

In [ ]:
# Folder with Lung Cancer Dataset of CT volumes 
LIDC_DBB_folder="//home/ensai/Documents/msd06-1-smart-data-project/rxtools/data/lungCT/LIDC-IDRI/"
# create .pylidcrc file in ~/
os.system("echo [dicom] > ~/.pylidcrc")
os.system(f'echo path={LIDC_DBB_folder} >> ~/.pylidcrc')
os.system("echo warn=True >> ~/.pylidcrc")

In [ ]:
### to generate the complete dataset, the code below has to be looped over all patients IDs
pid = 'LIDC-IDRI-0001' # patient ID 
scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == pid).first() # scan class instance for patient ID

In [ ]:
vol = scan.to_volume() # creating numpy array with scan volume
print(f'Scan: {scan}')
print(f'Scan spacing: {scan.pixel_spacing}')

vol_shape=vol.shape

print(f'Volume shape: {vol_shape}')


`pixel_spacing` - attribute of scan object. 

float – Dicom attribute (0028,0030). This is normally two values. All scans in the LIDC have equal resolutions in the transverse plane, so only one value is used here.

## Annotation clustering

The scan can have multiple annotations, but we several annotation can refer to the same nodule. We need to use annotation clustering to differentiate multiple nodules. This can be determined using the `pylidc.Scan.cluster_annotations()` method, which uses a distance function to create an adjancency graph to determine which annotations refer to the same nodule in a scan

In [ ]:
# USe annotation clustering to differentiate nodules

nods = scan.cluster_annotations()

print(f"{scan} has {len(nods)} nodules.")

for i, nod in enumerate(nods):
    print(f"Nodule {i}: has {len(nod)} annotation")
    for ann in nod: 
        print(f"\t{ann}")

scan.visualize(annotation_groups=nods)
plt.show()

### Visualize mid slice of CT volume

In [ ]:
plt.imshow(vol[:, :, round(vol_shape[2]/2)].squeeze(), cmap='gray')

## Compute x-ray projections and create Annotations Gold Standard

Compute x-ray projections from CT volumes. Also, create a volume with the annotations.

In [ ]:
### parameters of the geometry of projection
# We use diff DRR made to have differentiable DRR from CT volumes. Diff DRR is customized for cone-beam CTs
# https://github.com/eigenvivek/DiffDRR

sdr = 750  ## source-to-detector radius 
height = 1024 ### size of the projected image
delx = (4*200)/height
spacing = np.array([scan.pixel_spacing,scan.pixel_spacing, scan.slice_spacing])
print(f"CT voxel resolution is {spacing} mm")

# Initialize the DRR module for generating synthetic X-rays
#device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Found torch device is {device}")

### poses of camera to create projections ### this provides the geometric constraint
### Diff DRR being created for a cone beam CT, lets try these
# Set the camera pose with rotation (yaw, pitch, roll) and translation (x, y, z)
#rotation = torch.tensor([[torch.pi, 0.0, torch.pi / 2]], device=device)
rot_x=0.0
rot_y=0.0
rot_z=3*torch.pi/2
rotation = torch.tensor([[rot_x, rot_y, rot_z]], device=device)

bx, by, bz = torch.tensor(vol.shape) * torch.tensor(spacing) / 2 ### center of the CT volume in mm
translation = torch.tensor([[bx, by, bz]], device=device) ### TO CHECK how this defines the camera pose according to the diffDRR package

In [ ]:

torch.cuda.empty_cache()
torch.cuda.set_per_process_memory_fraction(0.8, 0)  # Use 80% of GPU memory

In [ ]:
## create drr object from CT
drr = DRR(
    vol,      # The CT volume as a numpy array
    spacing,     # Voxel dimensions of the CT
    sdr=sdr,   # Source-to-detector radius (half of the source-to-detector distance)
    height=height,  # Height of the DRR (if width is not seperately provided, the generated image is square)
    delx=delx,    # Pixel spacing (in mm)
).to(device)



### create drr images using drr object for N_views. Each view is a rotation around the x axis of 360°/N_views from 0°
N_views = 1
for idx_x in range(N_views):
    rad_x = idx_x*2*torch.pi/N_views
    
    rotation = torch.tensor([[rad_x, rot_y , rot_z]], device=device) 

    # 📸 Also note that DiffDRR can take many representations of SO(3) 📸
    # For example, quaternions, rotation matrix, axis-angle, etc...
    img = drr(rotation, translation, parameterization="euler_angles", convention="ZYX")
    ### here use your favorite tool to save the created xray images.
    ##visualisation
    plt.figure()
    plot_drr(img, ticks=False)
    plt.show()

In [ ]:
ann=scan.annotations ## annotation in scan
### Loop over annotations for CT volumes
for idx, a in enumerate(ann):
    ## create volume for the annotation
    vol_ann = np.zeros(vol.shape)
    vol_ann[a.bbox()] = 1  ### a.bbox() contains the gold standard (3D bounding box) of the localisation of each annotation 
    ##  vol_ann is a binary volume the same size as the CT images, where all pixels are zero outside the box containing the indications and 1 inside the box
    #
    ## create drr object for the volume of the anotation
    ## create drr object from CT
    drr = DRR(
        vol_ann,   # The binary volume with the annotations
        spacing,   # Voxel dimensions of the volume
        sdr=sdr,   # Source-to-detector radius (half of the source-to-detector distance)
        height=height,  # Height of the DRR (if width is not seperately provided, the generated image is square)
        delx=delx,    # Pixel spacing (in mm)
    ).to(device)
    #
    ## generates the xray images 
    for idx_x in range(N_views):
        rad_x = idx_x*2*torch.pi/N_views
        print(rad_x)
        rotation = torch.tensor([[rad_x, rot_y , rot_z]], device=device) 

        # 📸 Also note that DiffDRR can take many representations of SO(3) 📸
        # For example, quaternions, rotation matrix, axis-angle, etc...
        img = drr(rotation, translation, parameterization="euler_angles", convention="ZYX")
        img[img>0] = 1
        ### here use your favorite tool to save the created xray images s.
        ##visualisation
        plt.figure()
        plot_drr(img, ticks=False)
        plt.show()

### Investigate the rotations and transformations of DRR

In [3]:
import matplotlib.pyplot as plt
import pyvista
import torch

from diffdrr.data import load_example_ct
from diffdrr.drr import DRR
from diffdrr.pose import convert
from diffdrr.visualization import drr_to_mesh, img_to_mesh, labelmap_to_mesh, plot_drr

pyvista.start_xvfb()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
     

# Read in the CT volume
subject = load_example_ct()

# Make a mesh from the CT volume
# ct = drr_to_mesh(subject, "surface_nets", threshold=225, verbose=True)
     


In [4]:

# Read in the CT volume
subject = load_example_ct()

# # Make a mesh from the CT volume
# ct = drr_to_mesh(subject, "surface_nets", verbose=True)
     


In [5]:
# Initialize the DRR module for generating synthetic X-rays
drr = DRR(subject, sdd=1020.0, height=200, delx=2.0).to(device)

# Make a pose
rot = torch.tensor([[45.0, 30.0, 0.0]], device=device) / 180 * torch.pi
xyz = torch.tensor([[0.0, 800.0, 0.0]], device=device)
pose = convert(rot, xyz, parameterization="euler_angles", convention="ZXY")
# plot_drr(drr(pose))
# plt.show()

In [ ]:
# ct = drr_to_mesh(subject, "surface_nets", threshold=225, verbose=True)

In [6]:
# Make a mesh from the camera and detector plane
camera, detector, texture, principal_ray = img_to_mesh(drr, pose)

# Make the plot
plotter = pyvista.Plotter()
# plotter.add_mesh(ct)
plotter.add_mesh(camera, show_edges=True, line_width=1.5)
plotter.add_mesh(principal_ray, color="lime", line_width=3)
plotter.add_mesh(detector, texture=texture)

# Render the plot
plotter.add_axes()
plotter.add_bounding_box()
plotter.show_bounds(grid="front", location="outer", all_edges=True)

# plotter.show()  # If running Jupyter locally
# plotter.show(jupyter_backend="server")  # If running Jupyter remotely
plotter.export_html("render.html")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/ensai/anaconda3/envs/rxtool/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_590125/2088177000.py", line 2, in <module>
    camera, detector, texture, principal_ray = img_to_mesh(drr, pose)
                                               ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ensai/anaconda3/envs/rxtool/lib/python3.11/site-packages/diffdrr/visualization.py", line 296, in img_to_mesh
    img = drr(pose, calibration)
          ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ensai/anaconda3/envs/rxtool/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ensai/anaconda3/envs/rxtool/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
